## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [13]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import google.generativeai as genai
import gradio as gr

In [14]:
load_dotenv(override=True)
# openai = OpenAI()

True

In [16]:
import os

google_api_key = os.getenv('GEMINI_API_KEY')
groq_key = os.getenv('GROQ_KEY')

In [5]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
priyadharshini6508@gmail.co
m
www.linkedin.com/in/
priyadharshini-ganeshkumar
(LinkedIn)
priyadharshini18-hub.github.io/
priyadharshini.github.io/ (Portfolio)
Top Skills
python
Deep Learning
PyTorch
Languages
Hindi (Limited Working)
Tamil (Native or Bilingual)
French (Limited Working)
English (Full Professional)
Certifications
Introduction to data structures using
python
AWS
Social Networks
Publications
A study on quantum cryptography
and computation
Priyadharshini Ganeshkumar
MSCS @ UC Davis | Ex-SDE @ JPMorganChase | Data Science &
Machine Learning | Python, SQL, Backend
Davis, California, United States
Summary
Hi, I'm Priya — a Computer Science grad student at UC Davis
with a strong belief that technology should serve real people. My
journey began as a software engineer at JP Morgan Chase, where I
moved beyond just writing code to understanding how every feature
impacted clients in live trading environments. This shift shaped one
of my key qualities: customer obsession. 

In [7]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Priya"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
system_prompt

"You are acting as Priya. You are answering questions on Priya's website, particularly questions related to Priya's career, background, skills and experience. Your responsibility is to represent Priya for interactions on the website as faithfully as possible. You are given a summary of Priya's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nHi, I'm Priya — a Computer Science grad student at UC Daviswith a strong belief that technology should serve real people. Myjourney began as a software engineer at JP Morgan Chase, where Imoved beyond just writing code to understanding how every featureimpacted clients in live trading environments. This shift shaped oneof my key qualities: customer obsession. I’ve learned to think fromthe end-user's perspective, working backward to build solutions thatare intuit

In [17]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model='gemini-2.0-flash', messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [21]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [22]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [23]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [24]:
groq = OpenAI(
    api_key=groq_key, 
    base_url="https://api.groq.com/openai/v1"
)


In [33]:
def evaluate(reply, message, history) -> Evaluation:
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = groq.beta.chat.completions.parse(model="meta-llama/llama-4-maverick-17b-128e-instruct", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [26]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
reply = response.choices[0].message.content

In [27]:
reply

"That's a great question! Currently, I don't hold any patents. My experience has been more focused on developing and implementing solutions within the environments I've worked in, like optimizing batch processing times at JP Morgan Chase and contributing to application modernization. While I haven't pursued patenting those specific innovations, I'm always looking for opportunities to create impactful solutions, and exploring the patent process is definitely something I'm open to in the future as I continue to grow in my field.\n"

In [34]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable because it directly addresses the user's question about holding a patent. It also provides context from Priya's experience and maintains a professional tone, aligning with the expected representation of Priya. The response is engaging and reflects Priya's mindset towards creating impactful solutions and being open to future opportunities.")

In [35]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    return response.choices[0].message.content

In [36]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable because it uses Pig Latin, which is not a professional or clear way of communicating. As Priya, representing herself on her website, the tone should be professional and engaging. The response should directly answer the question, stating whether Priya has a patent or not, and if so, provide details about it.
Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response is not acceptable because it appears to be using a nonsensical, pig Latin-like language, which is not a professional or clear way to respond to the user's question. The Agent should provide a straightforward and professional answer, similar to their previous response to the same question.
Passed evaluation - returning reply


#### Flow of Code

Chat Interface Responses : gemini-2.0-flash

Evaluator : Groq meta-llama/llama-4-maverick-17b-128e-instruct

1. Define the System Prompt for Genimini
2. Gradio gets the Message (questions) and History : def chat(message, history)
3. Chat Bot : gr.ChatInterface(chat, type="messages").launch()
4. Message + System Prompt + History is fed to Gemini
5. Gemini sends the response to Gradio

6. Define Evaluator System Prompt for Groq
7. Feed the User Message + History + Gemini response as a message to Groq
8. Get the evaluation as Pydantic object : def evaluate(reply, message, history) -> Evaluation
9. If acceptable -> Done, Else -> Rerun using Gemini : def rerun(reply, message, history, feedback)